# The Basics

*Temporary Style Settings here*
<style>
li {
    list-style: disc;
    margin-left: 2em;
}
li p {
    list-style: disk;
    line-height: normal;
    margin-bottom: 0;
}
table p {
    list-style: disk;
    line-height: normal;
    margin: 0 2em;
    text-align: left;
}
</style>

## Section Components

A section defines a continuous portion of a text stream or other iterable.
A Section instance is the set of definitions for managing a sequence. the same 
instance can be used repeatedly on different sequence inputs or even on 
different portions of the same sequence.

The section definition may include:
* Boundary definitions for identifying the section's start and end.
* Formatting or processing instructions for manipulating section items.
* A merge method, which combines all of the processed items into a 
  single item such as a list.


The code in each stage of this example is independent of previous stages. 
Copy the one you want without tracking back to previous stages.

## Boundary Definitions

Every section has a *start_section* and an *end_section* which define the 
boundaries of the section.  The *start_section* and *end_section* each contain 
one or more *SectionBreak* objects

A *SectionBreak* is built from a *sentinel* and two optional modifiers:
* location
* offset

The *SectionBreak* can also be given a name, that can be used to identify which
*SectionBreak* was triggered when a section boundary definition contains 
multiple *SectionBreaks*.  See **Advanced Section Breaks** for more details. 

### Boundary Defaults

If *start_section* is not explicitly defined it defaults to `True` 
(*AlwaysBreak*), indicating the section begins with the first item in the 
supplied sequence.

If *end_section* is not explicitly defined it defaults to `False`, 
(*NeverBreak*)indicating the section continues through the last item in the 
supplied sequence.


In [1]:
from sections import Section
from pprint import pprint

example_sequence = [
    'First line',
    'Second line',
    'Third line',
    'Fourth line'
    ]

default_section = Section()
pprint(default_section.read(example_sequence))

['First line', 'Second line', 'Third line', 'Fourth line']


The section above uses the default boundary definitions (and default everything else for that matter).
With the default boundary definitions, every item in *example_sequence* is included in the section.

### Simple Text Boundary Definitions
The simplest boundary definitions is just a text string contained in the start or end item.

In [2]:
from sections import Section
from pprint import pprint

example_sequence = [
    'First line',
    'Second line',
    'Third line',
    'Fourth line'
    ]
text_boundary_section = Section(start_section='Second', 
                                end_section='Fourth')

pprint(text_boundary_section.read(example_sequence))

['Second line', 'Third line']


* `start_section='Second'` Causes the section to begin with the line containing *'Second'*
* `end_section='Fourth'` Causes the section to end before the line containing *'Fourth'*

### Adding a *location* Modifier to a String Boundary Definition 
By default, when a string is used to define a boundary, any occurrence of that 
string within a sequence item will trigger the boundary.
For example:

In [3]:
from sections import Section
from pprint import pprint


example_sequence = [
    'First line',
    'Second line',
    'Third line',
    'Fourth line'
   ]
text_boundary_section = Section(start_section='S', 
                                end_section='Fo')

pprint(text_boundary_section.read(example_sequence))

['Second line', 'Third line']


<b>S</b> was found in '<b>S</b>econd line' and
<b>Fo</b> was found in '<b>Fo</b>urth line'.

But what if as <b>S</b> was found in an earlier line?  For Example:

In [4]:
from sections import Section, SectionBreak
from pprint import pprint


example_sequence = [
    'Text String to be ignored',
    'First line',
    'Second line',
    'Third line',
    'Fourth line',
    'Even more text to be ignored', 
   ]

text_boundary_section = Section(start_section='S', 
                                end_section='Fo')

pprint(text_boundary_section.read(example_sequence))

['Text String to be ignored', 'First line', 'Second line', 'Third line']


The <b>S</b> in '<b>S</b>tring' triggered the boundary.

The optional `location` modifier allows you to specify where in the item string 
to search for the specified text.

The location argument can be one of:
<table>
<thead><tr><th><code>location</code> Value</th><th>Search Method</th></tr></thead>
<tbody>
<tr><td>'IN'</td><td><code>text in item</code></td></tr>
<tr><td>'START'</td><td><code>item.startswith(text)</code></td></tr>
<tr><td>'END'</td><td><code>item.endswith(text)</code></td></tr>
<tr><td>'FULL'</td><td><code>item == text</code></td></tr>
</tbody></table>

The section definition can then be given as:

In [5]:
from sections import Section, SectionBreak
from pprint import pprint


example_sequence = [
    'Text String to be ignored',
    'First line',
    'Second line',
    'Third line',
    'Fourth line',
    'Even more text to be ignored', 
   ]

text_boundary_section = Section(start_section=('S', 'START'),
                                end_section='Fo')

pprint(text_boundary_section.read(example_sequence))

['Second line', 'Third line']


Notice that `start_section=('S', 'START')` is now being set as a tuple.

An alternative, and perhaps clearer way to write the same this is to explicitly 
pass a `SectionBreak` object to `start_section` like this:

In [6]:
from sections import Section, SectionBreak
from pprint import pprint


example_sequence = [
    'Text String to be ignored',
    'First line',
    'Second line',
    'Third line',
    'Fourth line',
    'Even more text to be ignored', 
   ]

text_boundary_section = Section(
    start_section=SectionBreak(sentinel='S', location='START'),
    end_section='Fo'
    )

pprint(text_boundary_section.read(example_sequence))

['Second line', 'Third line']


For more information on the `location` modifier see *Advanced Section Breaks* 

In [7]:
from sections import Section, SectionBreak
from pprint import pprint


example_sequence = [
    'Text String to be ignored',
    'First line',
    'Second line',
    'Third line',
    'Fourth line',
    'Even more text to be ignored', 
   ]

text_boundary_section = Section(start_section='S', 
                                end_section='Fo')

pprint(text_boundary_section.read(example_sequence))

['Text String to be ignored', 'First line', 'Second line', 'Third line']


### Adding an *Offset* to a Boundary Definition
By default section boundaries occur *before* the item that triggers the 
boundary condition.
This can be changed using the optional `break_offset` argument.

The two most popular `break_offset` options are:
<table>
<thead><tr><th><code>break_offset</code> Value</th><th>Effect</th></tr></thead>
<tbody>
<tr><td>'After'</td>
<td>The SectionBreak occurs between the item that triggered the boundary and the 
next item.</td></tr>
<tr><td>'Before' <i>(the default)</i></td>
<td>The SectionBreak occurs is just before the item that triggered the 
boundary</td></tr>
</tbody></table>

For example, to include *'Fourth line'* in the section without knowing what 
comes next:

In [8]:
from sections import Section, SectionBreak
from pprint import pprint


example_sequence = [
    'Text String to be ignored',
    'First line',
    'Second line',
    'Third line',
    'Fourth line',
    'Even more text to be ignored', 
   ]

text_boundary_section = Section(
    start_section=SectionBreak(sentinel='S', location='START'),
    end_section=SectionBreak(sentinel='Fo', break_offset='After')
    )

pprint(text_boundary_section.read(example_sequence))

['Second line', 'Third line', 'Fourth line']


For more information on the `break_offset` modifier see *Advanced Section Breaks* 

### Defining a section boundary with a function.
A function can also be used to define a boundary.

In it's simplest form, the function should accept a single argument of the 
same type as the input sequence's items and return a boolean.  When the function
returns `True` a boundary is triggered.

For example:

In [9]:
from sections import Section, SectionBreak


numeric_sequence = [i for i in range(1,10)]
print('The sequence is:\t', numeric_sequence)


def multiple_of_three(num):
    return num % 3 == 0  # True if num is a multiple of 3


function_boundary_section = Section(start_section=multiple_of_three,
                                    end_section=multiple_of_three)

print('The section is: \t', function_boundary_section.read(numeric_sequence))

The sequence is:	 [1, 2, 3, 4, 5, 6, 7, 8, 9]
The section is: 	 [3, 4, 5]


In the example above:
* The sequence consists of a list of integers from 1 to 9.
* The function `multiple_of_three` returns `True` when it's input is a multiple 
of 3 and returns `False` otherwise.
* Both `start_section` and `end_section` are set as the `multiple_of_three` 
function.
* The section starts with the first multiple of three (3) and ends before the 
second multiple of three (6).

For more information on section boundaries see *Advanced Section Breaks* and 
*Using Context*